In [ ]:
import pandas as pd
from io import StringIO

def convert_df_to_json(df):
    json_data = df.to_json(orient="columns")
    return json_data

def preprocess_data(csv_data):
    max_dt = pd.to_datetime("1900-01-01 12:00:00")
    min_dt = pd.to_datetime("2099-01-01 12:00:00")
    for key, value in csv_data.items():
        if isinstance(value, list):
            continue # TODO return Error because data is missing
        value["Timestamp"] = pd.to_datetime(value["Timestamp"])
        value = value.set_index("Timestamp").drop("Serial Number", axis=1)
        value = value.add_prefix(f"{key}_")
        if max_dt < value.index.max():
            max_dt = value.index.max()
        if min_dt > value.index.min():
            min_dt = value.index.min()
        csv_data[key] = value

    df = pd.DataFrame({'DateTime':pd.date_range(start=min_dt, end=max_dt, freq='1min')})
    df = df.set_index('DateTime')

    df["basal_Insulin Delivered (U)"] = df["basal_Rate"]/60 # df["basal_Duration (minutes)"] *
    df["basal_Insulin Delivered (U)"] = df["basal_Insulin Delivered (U)"].ffill()
    df['bolus_Trigger'] = 0
    df['bolus_Trigger'].loc[(df['bolus_Carbs Input (g)'] == 0)] = 1 # automatický bolus
    df['bolus_Trigger'].loc[(df['bolus_Carbs Input (g)'] != 0) & (df['bolus_Carbs Input (g)'].notna()) & (df['bolus_Carbs Ratio'] != 0) & (df['bolus_Carbs Ratio'].notna())] = 2 # manuální bolus s jídlem
    df['bolus_Trigger'].loc[(df['bolus_Carbs Input (g)'] == 0) & (df['bolus_Carbs Input (g)'].notna()) & (df['bolus_Carbs Ratio'] != 0) & (df['bolus_Carbs Ratio'].notna())] = 3 # manuální bolus bez jídla
    df["bolus_Insulin Delivered (U)"] = df["bolus_Insulin Delivered (U)"].fillna(0)
    df["cgm_CGM Glucose Value (mmol/l)"] = df["cgm_CGM Glucose Value (mmol/l)"].interpolate(method='time')
    df["HypoEvent"] = 0
    time_window = 120
    for idx, row in df.reset_index().iterrows():
        if row["bolus_Trigger"] != 0:
            if any(df["cgm_CGM Glucose Value (mmol/l)"][idx:idx+time_window] < 3.9):
                df["HypoEvent"][idx] = 1
    df["Time"] = (df.index).time
    return df

def process_to_csv(data):
    csv_data = data['csvData']
    df_list = []

    for key, value in csv_data.items():
        for file in value:
            csv_content = "\n".join(file.split("\r\n")[1:])
            df = pd.read_csv(StringIO(csv_content))
            df_list.append(df)
        if not(df_list):
            continue
        combined_df = pd.concat(df_list, ignore_index=True)
        csv_data[key] = combined_df
        df_list = []
    return csv_data

def load_data():
    csv_dict = {
    "basal" : pd.read_csv("IT 30 ENG/Insulin data/basal_data_1.csv",skiprows=1)
    ,"bolus" : pd.read_csv("IT 30 ENG/Insulin data/bolus_data_1.csv",skiprows=1)
    ,"insulin" : pd.read_csv("IT 30 ENG/Insulin data/insulin_data_1.csv",skiprows=1)
    ,"alarms": pd.read_csv("IT 30 ENG/alarms_data_1.csv",skiprows=1)
    ,"bg" : pd.read_csv("IT 30 ENG/bg_data_1.csv",skiprows=1)
    ,"cgm" : pd.read_csv("IT 30 ENG/cgm_data_1.csv",skiprows=1)
    }
    return csv_dict

# data = {'csvData':
# 	{'basal': []
# 	, 'bolus': []
# 	, 'insulin': [
# 'Name:,Date Range:2024-06-02 - 2024-07-01\r\nTimestamp,Total Bolus (U),Total Insulin (U),Total Basal (U),Serial Number\r\n2024-07-01 21:50,3.64,15.23,11.59,935953\r\n2024-06-30 23:58,9.11,20.11,11.0,935953\r\n2024-06-29 23:58,4.16,15.43,11.27,935953\r\n2024-06-28 23:58,9.62,21.1,11.48,935953\r\n2024-06-27 23:58,7.27,19.92,12.65,935953\r\n2024-06-26 23:58,14.72,25.64,10.92,935953\r\n2024-06-25 23:58,7.79,18.95,11.16,935953\r\n2024-06-24 23:58,6.9,19.18,12.28,935953\r\n2024-06-23 23:58,12.9,21.65,8.75,935953\r\n2024-06-22 23:58,6.21,17.28,11.07,935953\r\n2024-06-21 23:58,10.32,20.01,9.69,935953\r\n2024-06-20 23:58,9.53,23.97,14.44,935953\r\n2024-06-19 23:58,5.67,17.28,11.61,935953\r\n2024-06-18 23:58,8.67,20.09,11.42,935953\r\n2024-06-17 23:58,5.41,18.47,13.06,935953\r\n2024-06-16 23:58,16.92,29.52,12.6,935953\r\n2024-06-15 23:58,11.46,22.43,10.97,935953\r\n2024-06-14 23:58,3.55,12.65,9.1,935953\r\n2024-06-13 23:58,6.44,15.87,9.43,935953\r\n2024-06-12 23:58,6.28,15.35,9.07,935953\r\n2024-06-11 23:58,10.98,25.01,14.03,935953\r\n2024-06-10 23:58,7.41,17.66,10.25,935953\r\n2024-06-09 23:58,9.73,22.15,12.42,935953\r\n2024-06-08 23:58,11.54,22.73,11.19,935953\r\n2024-06-07 23:58,8.83,20.24,11.41,935953\r\n2024-06-06 23:58,7.59,17.66,10.07,935953\r\n2024-06-05 23:58,6.02,19.09,13.07,935953\r\n2024-06-04 23:58,8.6,21.0,12.4,935953\r\n2024-06-03 23:58,14.96,27.5,12.54,935953\r\n2024-06-02 23:58,12.6,25.25,12.65,935953\r\n'
# , 'Name:,Date Range:2024-06-02 - 2024-10-10\r\nTimestamp,Total Bolus (U),Total Insulin (U),Total Basal (U),Serial Number\r\n2024-10-10 21:50,3.64,15.23,11.59,935953\r\n2024-06-30 23:58,9.11,20.11,11.0,935953\r\n2024-06-29 23:58,4.16,15.43,11.27,935953\r\n2024-06-28 23:58,9.62,21.1,11.48,935953\r\n2024-06-27 23:58,7.27,19.92,12.65,935953\r\n2024-06-26 23:58,14.72,25.64,10.92,935953\r\n2024-06-25 23:58,7.79,18.95,11.16,935953\r\n2024-06-24 23:58,6.9,19.18,12.28,935953\r\n2024-06-23 23:58,12.9,21.65,8.75,935953\r\n2024-06-22 23:58,6.21,17.28,11.07,935953\r\n2024-06-21 23:58,10.32,20.01,9.69,935953\r\n2024-06-20 23:58,9.53,23.97,14.44,935953\r\n2024-06-19 23:58,5.67,17.28,11.61,935953\r\n2024-06-18 23:58,8.67,20.09,11.42,935953\r\n2024-06-17 23:58,5.41,18.47,13.06,935953\r\n2024-06-16 23:58,16.92,29.52,12.6,935953\r\n2024-06-15 23:58,11.46,22.43,10.97,935953\r\n2024-06-14 23:58,3.55,12.65,9.1,935953\r\n2024-06-13 23:58,6.44,15.87,9.43,935953\r\n2024-06-12 23:58,6.28,15.35,9.07,935953\r\n2024-06-11 23:58,10.98,25.01,14.03,935953\r\n2024-06-10 23:58,7.41,17.66,10.25,935953\r\n2024-06-09 23:58,9.73,22.15,12.42,935953\r\n2024-06-08 23:58,11.54,22.73,11.19,935953\r\n2024-06-07 23:58,8.83,20.24,11.41,935953\r\n2024-06-06 23:58,7.59,17.66,10.07,935953\r\n2024-06-05 23:58,6.02,19.09,13.07,935953\r\n2024-06-04 23:58,8.6,21.0,12.4,935953\r\n2024-06-03 23:58,14.96,27.5,12.54,935953\r\n2024-06-02 23:58,12.6,25.25,12.65,935953\r\n'
# 			]
# 	, 'alarms': []
# 	, 'bg': []
# 	, 'cgm': []}
# }

csv_data = process_to_csv(data)
df = preprocess_data(csv_data)
json_data = convert_df_to_json(df)

KeyError: 'basal_Rate'